In [4]:
import numpy as np
import pandas as pd

In [5]:
# 6.2 이진 데이터 형식

In [6]:
# 데이터를 효율적으로 저장하는 가장 손쉬운 방법은 파이썬에 기본으로 내장되어 있는 pickle 직렬화를 사용해 데이터를 이진 형식으로 저장하는 것이다. 

In [7]:
# 편리하게도 pandas 객체는 모두 pickle을 이용해서 데이터를 저장하는 to_pickle 메서드를 가지고 있다.

In [8]:
frame = pd.read_csv("examples/ex1.csv")

In [9]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
frame.to_pickle("examples/frame_pickle")

In [11]:
# pickle로 직렬화된 객체는 내장 함수인 pickle로 직접 불러오거나 아니면 좀 더 편리한 pickle 함수인 pandas.read_pickle 메서드를 이용하여 불러올 수 있다.

In [12]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
# 주의! pickle은 오래 보관할 필요가 없는 데이터일 경우에만 추천한다. 오랜 시간이 지나도 안정적으로 데이터를 저장할 거라고 보장하기 힘든 문제가 있기 때문이다.

In [14]:
# pandas는 HDF5와 Message-Pack, 두 가지 바이너리 포맷을 지원한다. 

In [15]:
# 다음 절에서 HDF5 예제를 살펴보겠지만 다양한 파일 형식이 실제 독자들의 분석 작업에 얼마나 더 적절한지 직접 살펴봐야할 필요가 있다.

In [16]:
# 다음과 같은 pandas 또는 NumPy 데이터를 위한 다른 저장 형식도 존재한다.

In [17]:
# Bcolz : Blocs 압축 알고리즘에 기반한 압축이 가능한 컬럼지향 바이너리 포맷이다.

In [18]:
# Feather : R 커뮤니티의 해들리 위컴과 저자가 함께 설계한 컬럼지향 파일 형식이다. Feather는 아파치 에로우의 메모리 포맷을 사용한다.

In [19]:
# 6.2.1 HDF5 형식 사용하기

In [20]:
# HDF5는 대량의 과학 계산용 배열 데이터를 저장하기 위해 고안된 훌륭한 파일 포맷이다.

In [21]:
# C 라이브러리로도 존재하며 자바, 줄리아, 매트랩, 그리고 파이썬 같은 다양한 다른 언어에서도 사용할 수 있는 인터페이스를 제공한다.

In [22]:
# HDF는 Hiearchical Data Format의 약자로 계층적 데이터 형식이라는 뜻이다. 각각의 HDF5 파일은 여러 개의 데이터셋을 저장하고 부가 정보를 기록할 수 있다.

In [23]:
# 보다 단순한 형식과 비교하면 HDF5는 다양한 압축 기술을 사용해서 실시간 압축을 지원하며 반복되는 패턴을 가진 데이터를 좀 더 효과적으로 저장할 수 있다.

In [24]:
# 메모리에 모두 적재할 수 없는 엄청나게 큰 데이터를 아주 큰 배열에서 필요한 작은 부분들만 효과적으로 읽고 쓸 수 있는 훌륭한 선택이다.

In [25]:
# PyTables나 h5py 라이브러리를 이용해서 직접 HDF5 파일에 접근하는 것도 가능하지만 pandas는 Series나 DataFrame 객체로 간단히 저장할 수 있는 고수준의 인터페이스를 제공한다.

In [26]:
# HDFStore 클래스는 사전처럼 작동하며 세밀한 요구 사항도 잘 처리해준다.

In [27]:
frame = pd.DataFrame({"a": np.random.randn(100)})

In [28]:
store = pd.HDFStore("mydata.h5")

In [29]:
store["obj1"] = frame

In [30]:
store["obj1_col"] = frame["a"]

In [31]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [32]:
# HDF5 파일에 포함된 객체는 파이썬 사전과 유사한 형식으로 사용 가능하다.

In [33]:
store["obj1"]

,a
0,-1.393931
1,0.890973
2,0.405025
3,-1.217998
4,1.382295
...,...
95,0.065271
96,0.367615
97,-0.881677
98,-1.247105


In [34]:
# HDFStore는 "fixed"와 "table" 두 가지 저장 스키마를 지원한다.

In [35]:
# "table" 스키마가 일반적으로 더 느리지만 아래와 같은 특별한 문법을 이용해 쿼리 연산을 지원한다.

In [36]:
store.put("obj2", frame, format="table")

In [37]:
store.select("obj2", where=["index >= 10 and index <= 15"])

,a
10,0.621478
11,1.558109
12,-0.158419
13,0.464220
14,1.056626
15,-2.087597


In [38]:
store.close()

In [39]:
# put은 명시적인 store["obj2"] = frame 메서드지만 저장 스키마를 지정하는 등의 다른 옵션을 제공한다. 

In [40]:
# pandas.read_hdf 함수는 이런 기능들을 축약해서 사용할 수 있다.

In [41]:
frame.to_hdf("mydata.h5", "obj3", format="table")

In [42]:
pd.read_hdf("mydata.h5", "obj3", where=["index < 5"])

,a
0,-1.393931
1,0.890973
2,0.405025
3,-1.217998
4,1.382295


In [43]:
# 만약 로컬 스토리지에서 엄청난 양의 데이터를 다뤄야 한다면 PyTables와 h5py를 살펴보고 목적에 맞는지 알아봐야 한다.

In [44]:
# 실제로 대부분의 데이터 분석 문제는 CPU보다는 IO 성능에 의존적이므로 HDF5 같은 도구를 사용하면 애플리케이션의 성능을 매우 향상시킬 수 있다.

In [45]:
# 6.4 앞부분은 코랩으로 진행했다.

In [46]:
# 6.4 데이터베이스와 함께 사용하기

In [47]:
# 비즈니스 관점에서 대부분의 데이터는 텍스트 파일이나 엑셀 파일로 저장하지 않고 SQL 기반의 관계형 데이터베이스(SQL, postgreSQL, MySQL)를 많이 사용한다.

In [48]:
# 데이터베이스는 보통 애플리케이션에서 필요한 성능이나 데이터 무결성 그리고 확장성에 맞춰서 선택하는 것이 일반적이다.

In [49]:
# SQL에서 데이터를 읽어 와서 DataFrame에 저장하는 것은 꽤 직관적이며 pandas에는 이 과정을 간결하게 해주는 몇 가지 함수가 있다.

In [50]:
# 한 예로 파이썬의 내장 sqlite3 드라이버를 사용해서 SQLite 데이터베이스를 이용할 수 있다.

In [51]:
import sqlite3

In [59]:
query = """
        CREATE TABLE test
        (a VARCHAR(20), b VARCHAR(20), 
        c REAL,         d INTEGER
        );"""

In [60]:
con = sqlite3.connect("mydata.sqlite")

In [61]:
con.execute(query)

OperationalError: table test already exists

In [62]:
con.commit()

In [63]:
# 이제 데이터를 몇개 입력한다.

In [66]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahasse", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]

In [67]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [68]:
con.executemany(stmt, data)

In [70]:
con.commit()

In [71]:
# 대부분의 파이썬 SQL 드라이버(PyODBC, psycopg2, MySQLdb, pymssql 등)는 테이블에 대해 select 쿼리를 수행하면 튜플 리스트를 반환한다.

In [72]:
cursor = con.execute("select * from test")

In [73]:
rows = cursor.fetchall()

In [74]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahasse', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [75]:
# 반환된 튜플 리스트를 DataFrame 생성자에 바로 전달해도 되지만, 컬럼 이름을 지정해주면 더 편하다. cursor의 description 속성을 활용하자.

In [76]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [77]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahasse,Florida,2.60,3
2,Sacramento,California,1.70,5


In [78]:
# 데이터베이스에 쿼리를 보내기 위해 매번 이렇게 하긴 어렵다. 

In [79]:
# 유명한 파이썬 SQL 툴킷인 SQLAlchemy(SQL알케미) 프로젝트는 SQL 데이터베이스 간의 일반적인 차이점을 추상화하여 제공한다. 

In [80]:
# pandas는 read_sql 함수를 제공하여 SQL알케미의 일반적인 연결을 이용해 쉽게 데이터를 읽을 수 있게 한다.

In [82]:
import sqlalchemy as sqla

In [83]:
db = sqla.create_engine("sqlite:///mydata.sqlite")

In [84]:
pd.read_sql("select * from test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahasse,Florida,2.60,3
2,Sacramento,California,1.70,5
